In [ ]:
pip install unidecode


In [ ]:
import geopandas as gpd
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import numpy as np
import unidecode
import pandas as pd
from rasterstats import zonal_stats

In [ ]:
print("Número de bandas:", raster.count)

In [ ]:
# Cargar shapefile
gdf = gpd.read_file("data/DISTRITOS.shp")

# Asegurar que los nombres de columnas estén en mayúsculas
gdf.columns = [col.upper() for col in gdf.columns]

# Limpiar tildes y pasar a mayúsculas en columnas clave
for col in ["DEPARTAMEN", "PROVINCIA", "DISTRITO"]:
    gdf[col] = gdf[col].apply(lambda x: unidecode.unidecode(x.upper()))

# Renombrar la columna GEOMETRY a geometry
gdf = gdf.rename(columns={"GEOMETRY": "geometry"})

# Asegurar que es GeoDataFrame y tiene CRS correcto
gdf = gpd.GeoDataFrame(gdf, geometry="geometry", crs="EPSG:4326")

# Renombrar la columna y convertirla a string
gdf = gdf.rename(columns={"IDDIST": "UBIGEO"})
gdf["UBIGEO"] = gdf["UBIGEO"].astype(str)

# Revisar resultado
display(gdf.head())


In [ ]:
# Cargar el GeoTIFF
src = rasterio.open("data/tmin_raster.tif")

# Ver cuántas bandas tiene
print("Número de bandas:", src.count)

# Iterar sobre las bandas (Band 1 = 2020, Band 2 = 2021, etc.)
for i in range(1, src.count + 1):
    band = src.read(i)
    year = 2020 + (i - 1)
    print(f"Banda {i} = Año {year}, tamaño: {band.shape}")

In [ ]:
#Vamos a verificar si todas las bandas estan con la escala correcta

print("📊 Valores por banda (para verificar escala):")
for i in range(1, raster.count + 1):
    band = raster.read(i)
    print(f"Banda {i} ({2019 + i}):")
    print(f"  Mín: {np.nanmin(band):.2f}")
    print(f"  Máx: {np.nanmax(band):.2f}")
    print("-" * 30)

#Podemos que sí, son temperaturas realistas

In [ ]:
results = []

for i in range(1, raster.count + 1):
    band = raster.read(i)
    year = 2019 + i  # banda 1 = 2020
    
    stats = zonal_stats(
        gdf,                  
        band,                 
        affine=raster.transform,
        nodata=raster.nodata,
        stats=["count", "mean", "min", "max", "std", "percentile_10", "percentile_90"],
        add_stats={"range": lambda x: np.nanmax(x) - np.nanmin(x)}
    )
    
    # Crear DataFrame limpio para esta banda
    df_band = gdf[["UBIGEO", "DEPARTAMEN", "PROVINCIA", "DISTRITO"]].copy()
    df_band["year"] = year
    df_band["count"] = [s["count"] for s in stats]
    df_band["mean"] = [s["mean"] for s in stats]
    df_band["min"] = [s["min"] for s in stats]
    df_band["max"] = [s["max"] for s in stats]
    df_band["std"] = [s["std"] for s in stats]
    df_band["p10"] = [s["percentile_10"] for s in stats]
    df_band["p90"] = [s["percentile_90"] for s in stats]
    df_band["range"] = [s["range"] for s in stats]
    
    results.append(df_band)

zonal_stats_df = pd.concat(results, ignore_index=True)
display(zonal_stats_df.head())


In [ ]:
# Guardar resultados en CSV
zonal_stats_df.to_csv("data/estadisticas_tmin.csv", index=False, encoding="utf-8")

print("✅ Archivo guardado en: data/estadisticas_tmin.csv")
